## A Dashboard for attractors
[Lázaro Alonso](https://lazarusa.github.io/Webpage/index.html)
___

An [attractor](https://en.wikipedia.org/wiki/Attractor#Strange_attractor) is a set of values to which a numerical system tends to evolve. An attractor is called a strange attractor if the resulting pattern has a fractal structure. This notebook shows how plot two-dimensional attractors of a variety of types, using the low-level GR `shadepoints` function. You can get this function by typing `add GR#master` into your packages manager. Additionally, for making slide bars and buttons, you will need to `add Interact`.

- https://gr-framework.org/index.html
- https://juliagizmos.github.io/Interact.jl/latest/

For example, a [clifford Attractor](http://paulbourke.net/fractals/clifford/) is a strange attractor defined by two iterative equations that determine the $(x_{n},y_{n})$ locations of discrete steps in the path of a particle across a 2D space, given a starting point $(x_{0},y_{0})$ and the values of four parameters $(a,b,c,d)$. Here, we will be considering only the following four cases. 

Attractor | Equation 
--- | --- 
*[Clifford](http://paulbourke.net/fractals/clifford/)* | \begin{align}
x_{n +1} &= \sin(a y_{n}) + c \cos(a x_{n})\\
y_{n +1} &= \sin(b x_{n}) + d \cos(b y_{n})
\end{align} 
 | 
*[Peter Jong](http://paulbourke.net/fractals/peterdejong/)* | \begin{align}
x_{n +1} &= \sin(a y_{n}) - \cos(b x_{n})\\
y_{n +1} &= \sin(c x_{n}) - \cos(d y_{n})
\end{align}
 |
*[Johnny Svensson](http://paulbourke.net/fractals/peterdejong/)* | \begin{align}
x_{n +1} &= d\sin(a y_{n}) - \sin(b y_{n})\\
y_{n +1} &= c\cos(a x_{n}) + \cos(b y_{n})
\end{align}
 |
*[Bedhead](https://softologyblog.wordpress.com/2017/03/04/2d-strange-attractors/)* | \begin{align}
x_{n +1} &= \sin(x_{n} y_{n}/b)y_{n} + \cos(bx_{n} -y_{n})\\
y_{n +1} &= x_{n} + \sin(y_{n})/b
\end{align} 

Moreover, besides the original mapping equations, the orbit will be slightly modified at each iteration as follows: 

\begin{equation}
x_{n+1}, y_{n+1} = x_{n}\cos(\theta_{n}),\, y_{n}\cos(\theta_{n}),\quad \theta_{n} = \theta_{n-1} +\delta\theta
\end{equation}

where $\delta\theta$ is a fixed small variation for each mapping. This is inspired in the [circular representation](https://nbviewer.jupyter.org/github/lazarusA/CodeSnippets/blob/master/CodeSnippetsPython/ScientificPlotBasic2DDensityEj2.ipynb) for the `logistic map`. 

For related work please visit: 
- [Clifford Attractor example in Julia](https://nbviewer.jupyter.org/github/lazarusA/CodeSnippets/blob/master/CodeSnippetsJulia/CliffordJuliaVersion.ipynb)
- [Clifford Attractor example in python](https://nbviewer.jupyter.org/github/lazarusA/CodeSnippets/blob/master/CodeSnippetsPython/ScientificPlotBasic2DDensityEj2.ipynb)

- https://anaconda.org/jbednar/clifford_attractor/notebook
- https://anaconda.org/jbednar/datashaderattractors/notebook
- http://paulbourke.net/fractals/

Let's load the packages needed. 

In [6]:
using GR, Interact, Pkg

For convenience, all definitions and examples are saved in a separated script. Let's include them:   

In [7]:
include("attractorsClifford.jl");

In [8]:
pkgsVersion()

Julia == 1.0.0
GR == 0.35.0+
IJulia == 1.13.0
Interact == 0.9.0


Now, the whole dashboard can be assembled with `@manipulate`. By default, you can choose different values for each parameter, and it will update automatically. But, if you **activate examples**, then you can only pick out from those, and the only free parameters left will be $N$(number of points), $\delta\theta$ and the colormap. To go back to choose all parameters you need to **deactivate examples**. 

**Note:** [colormaps in GR](https://gr-framework.org/colormaps.html) are usually called with an integer.

In [9]:
ui = @manipulate throttle = 0.1 for 
    a = slider(-2.0:0.01:2.0, value = 1.7, label = "a"), 
    b = slider(-2.0:0.01:2.0, value = 1.7, label = "b"),
    c = slider(-2.0:0.01:2.0, value = .6, label = "c"),
    d = slider(-2.0:0.01:2.0, value = 1.2, label = "d"), 
    δθ = slider(0.0:0.0001:0.1, value = 0.001, label = "δθ"),
    n = slider([500000, 1000000, 5000000, 10000000], value = 1000000, label = "N"),
    attractor = dropdown([Clifford, De_Jong, Svensson, Bedhead], label = "Attractor"),
    cmap = slider(0:47, label="color map", value = 8),
    slideExample = slider(1:6, label=">>"),
    examples = toggle("Examples", value = false),
    cmapr = toggle("cmap_r", value = true)
    
    signo = cmapr == true ? 1 : -1
    setcolormap(signo*cmap)

    if examples == true
        a, b, c, d = attractorExamples[attractor][slideExample]
        x, y = trajectory(attractor, 0.1, 0.1, a, b, c, d, δθ, n)
    else
        x, y = trajectory(attractor, 0.1, 0.1, a, b, c, d, δθ, n)
    end
    xmini = ceil(minimum(x))
    xmaxi = ceil(maximum(x))
    ymini = ceil(minimum(y))
    ymaxi = ceil(maximum(y))
    setviewport(0.1, 0.95, 0.1, 0.95)
    setwindow(xmini-1, xmaxi, ymini-1, ymaxi)
    setcharheight(0.022)
    axes2d(.5, .5, xmini -1, ymini-1, Int64(xmaxi), Int64(ymaxi), -0.005)
    settextfontprec(105, 0)
    shadepoints(x, y, dims = (1200, 1200), xform= 5)
    GR.show()
    end
@layout! ui dom"div"(
    hbox(vbox(vskip(1em), :attractor, hbox(:examples, hskip(1em), :cmapr),
            :slideExample, :cmap, :n, :δθ, :a, :b, :c, :d),
        observe(_)))

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:a=>Widget{:slider,Float64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:index=>Observable{Any} with 2 listeners. Value:
371,:formatted_vals=>Observable{Any} with 1 listeners. Value:
["-2.0", "-1.99", "-1.98", "-1.97", "-1.96", "-1.95", "-1.94", "-1.93", "-1.92", "-1.91"  …  "1.91", "1.92", "1.93", "1.94", "1.95", "1.96", "1.97", "1.98", "1.99", "2.0"],:formatted_value=>Observable{String} with 1 listeners. Value:
"1.7",:value=>Observable{Float64} with 2 listeners. Value:
1.7), Observable{Float64} with 2 listeners. Value:
1.7, Scope("knockout-component-2aec616e-3f76-474b-9bbf-df1db5cac38a", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["a"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>401,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("formatted_vals"=>(Observable{Any} with 1 listeners. Value:
["-2.0", "-1.99", "-1.98", "-1.97", "-1.96", "-1.95", "-1.94", "-1.93", "-1.92", "-1.91"  …  "1.91", "1.92", "1.93", "1.94", "1.95", "1.96", "1.97", "1.98", "1.99", "2.0"], nothing),"changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"formatted_value"=>(Observable{String} with 1 listeners. Value:
"1.7", nothing),"index"=>(Observable{Any} with 2 listeners. Value:
371, nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/lazaro/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/lazaro/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/lazaro/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/lazaro/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/lazaro/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("formatted_vals"=>Any[JSString("(function (val){return (val!=this.model[\"formatted_vals\"]()) ? (this.valueFromJulia[\"formatted_vals\"]=true, this.model[\"formatted_vals\"](val)) : undefined})")],"_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"formatte

Among several things that I like about this approach is that the aggregation of points can be easily mappped back to the _original coordinated system_. However, to really performed statistically analysis over data, an **easy access** to this **aggregation array** is requiered. Also, it will be nice if the `xform` option in the function `shadepoints` could be specified by the user. 

Note: `colorcet`  is not implemented yet.

For a simpler implementation of [`shadepoints`](https://twitter.com/josef_heinen/status/1058381267752939521) please visit  [Josef Heinen](https://twitter.com/josef_heinen) [repo](https://github.com/jheinen/GR.jl/blob/master/examples/clifford_attractor.ipynb). It seems that now he is using a new function called `shade`(still in progress). 